# Feature Engineering and EDA

In [2]:
import numpy as np
import pandas as pd
import pickle
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.tokenize import sent_tokenize
from textblob import TextBlob

In [3]:
file = open('articles.p', 'rb')      
df = pickle.load(file)
file.close()

In [ ]:
df['b_tokens'] = df['body'].map(lambda x: sent_tokenize(x)[:10])

In [ ]:
df2 = df.drop(columns=['url', 'body'])
df2['b_tokens'] = df['b_tokens'].map(lambda x: ' '.join(x))

In [ ]:
drop_indices = df2.loc[df2['date'] < pd.Timestamp(2019, 3, 15)].index
df2.drop(index=drop_indices, inplace=True)

In [ ]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
hl_sample = df2['headline'][849]
print('original document: ')
words = []
for word in hl_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(hl_sample))